# Proposal Prep

This notebook is used to generate the figures shown in the proposal outline.

In [ ]:
%matplotlib widget
import datetime as dt
import json

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.units as munits
import numpy as np
import pandas as pd
from pyts.image import RecurrencePlot

from sklearn.preprocessing import StandardScaler

converter = mdates.ConciseDateConverter()
munits.registry[np.datetime64] = converter
munits.registry[dt.date] = converter
munits.registry[dt.datetime] = converter

In [ ]:
with open('../ICME_WP4_V10.json', 'r') as fobj:
    json_data = json.load(fobj)
    
df = pd.DataFrame(json_data['data'], columns=json_data['columns'])

In [ ]:
df.head()

In [ ]:
stereo_a_df = df[df['SC_INSITU'].str.contains('STEREO-A')]
stereo_b_df = df[df['SC_INSITU'].str.contains('STEREO-B')]

In [ ]:
stereo_a_icme_times = stereo_a_df.loc[:,['ICME_START_TIME','ICME_END_TIME']]
stereo_b_icme_times = stereo_b_df.loc[:,['ICME_START_TIME','ICME_END_TIME']]

In [ ]:
stereo_a_icme_times['ICME_START_TIME'].min(), stereo_a_icme_times['ICME_START_TIME'].max()

In [ ]:
stereo_a_icme_times.head(10)

In [ ]:
stereo_b_icme_times

In [ ]:
def read_stereo_datasets(fname):
    """Function for reading in stereo datasets"""
    with open(fname, 'r') as fobj:
        lines = fobj.readlines()

    colnames = lines[0].split()
    tmp = lines[1].split()
    units = []
    units.append(' '.join(tmp[:2]))
    units += tmp[2:]

    for col, unit in zip(colnames, units):
        print(col, unit)
        
    data = []
    index = []
    for line in tqdm(lines[2:]):
        lsplit = line.split()
        index.append(dt.datetime.strptime(' '.join(lsplit[:2]), '%d-%m-%Y %H:%M:%S.%f'))
        data.append(list(map(float, lsplit[2:])))
    
    df = pd.DataFrame(data, columns=colnames[1:], index=pd.DatetimeIndex(index))
    return df

In [ ]:
df = pd.read_csv('/Users/ndmiles/Downloads/sta_mag_rtn_20070501T000000_20070531T000000_1m/sta_mag_rtn_20070501T000000_20070531T000000_1m.csv')

In [ ]:
df.describe()

In [ ]:
filtered_df = df[(df['BR A'].lt(1e31)) & (df['Np'].astype(float).lt(1e31))]

In [ ]:
filtered_df.index = pd.DatetimeIndex(filtered_df['Time'])

In [ ]:
import numpy as np

In [ ]:
filtered_df['B_mag'] = np.sqrt(
    filtered_df['BR A']**2 + filtered_df['BT A']**2 + filtered_df['BN A']**2
)

In [ ]:
filtered_df.plot(y=['BR A', 'BT A', 'BN A', 'B_mag'])

In [ ]:
scaled_Br = StandardScaler().fit_transform(filtered_df['BT A'].values.reshape(-1,1)).flatten()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(filtered_df.index, scaled_Br)

In [ ]:
scaled_Br

In [ ]:
delta = dt.timedelta(days=5)
icme_cut = slice(
    dt.datetime(2007,5,25) - delta, 
    dt.datetime(2007,5,25) + delta
)
sir_cut = slice(
    dt.datetime(2007, 5, 8) - delta,
    dt.datetime(2007, 5, 8) + delta
)
columns_to_plot = ['BR A', 'BT A', 'BN A', 'B_mag', 'Np', 'Vp', 'Tp']

In [ ]:
for col in columns_to_plot:
    filtered_df[col] = filtered_df[col].astype(float)

In [ ]:
def make_recurrence_plots(filtered_df, cut, cols):
    fig, ax = plt.subplots(nrows=len(cols), ncols=2, sharex='col', figsize=(6,10))
    rp = RecurrencePlot(threshold='point', percentage=20)
    resampled_df = filtered_df.resample('5min').mean()
    resampled_df = resampled_df.dropna()
    for i,col in enumerate(cols):
        data = resampled_df[cut]
        data_scaled = StandardScaler().fit_transform(data[col].values.reshape(-1,1)).flatten()

        ax[i, 0].plot(data[col].index, data_scaled, lw=0.7)
        ax[i, 0].set_ylabel(col)
        ax[i, 0].yaxis.set_major_locator(plt.MaxNLocator(5))
        data_rp = rp.fit_transform(data_scaled.reshape(1,-1))
        ax[i, 1].imshow(data_rp[0], cmap='binary', origin='lower', aspect='equal')
        ax[i, 1].tick_params(axis='both', which='both', bottom=False, left=False,labelbottom=False, labelleft=False)
#         if col == 'Tp':
#             ax[i, 0].set_yscale('log')

    
    ax[0,0].set_title('Time Series')
    ax[0,1].set_title('Image Representation')
        
    return fig, ax

In [ ]:
icme_fig, icme_ax = make_recurrence_plots(filtered_df, icme_cut, columns_to_plot)
icme_fig.suptitle(t='STEREO-A ICME', x=0.5, y=0.95)
# icme_fig.savefig('icme_example.jpg',format='jpg', dpi=250, bbox_inches='tight')

In [ ]:
sir_fig, sir_ax = make_recurrence_plots(filtered_df, sir_cut, columns_to_plot)
sir_fig.suptitle(t='STEREO-A SIR', x=0.5, y=0.95)
sir_fig.savefig('example_sir.jpg', format='jpg', dpi=250, bbox_inches='tight')